# Valuing Caps and Floors

We value caps and floors using Black's model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [14]:
from financepy.finutils import *
from financepy.products.libor.FinLiborDeposit import FinLiborDeposit
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.products.libor.FinLiborCapFloor import *
from financepy.products.libor import FinLiborCurve

## Building a Libor Curve

In [15]:
valuationDate = FinDate(6, 6, 2018)

In [16]:
spotDays = 0
settlementDate = valuationDate.addWorkDays(spotDays)
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA

depos = []
depo = FinLiborDeposit(settlementDate, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "2M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "9M", 0.0230, depoDCCType); depos.append(depo)

In [17]:
accrual = FinDayCountTypes.THIRTY_360
freq = FinFrequencyTypes.SEMI_ANNUAL
longEnd = FinDateGenRuleTypes.BACKWARD

spotDays = 2
settlementDate = valuationDate.addWorkDays(spotDays)

swaps = []
swap = FinLiborSwap(settlementDate, "1Y", 0.0250, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "2Y", 0.0255, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "3Y", 0.0260, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", 0.0265, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", 0.0270, freq, accrual); swaps.append(swap)

In [18]:
liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

## Creating a Cap and Floor

In [19]:
capType = FinLiborCapFloorTypes.CAP
floorType = FinLiborCapFloorTypes.FLOOR

In [20]:
strikeRate = 0.02

In [21]:
cap = FinLiborCapFloor(settlementDate, "2Y", capType, strikeRate)
flr = FinLiborCapFloor(settlementDate, "2Y", floorType, strikeRate)

In [22]:
print(cap)

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorTypes.CAP
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


In [23]:
print(flr)

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorTypes.FLOOR
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


# Valuation

### Black's Model

We start with Black's model with 25% volatility

In [24]:
model = FinModelBlack(0.25)

In [25]:
cap.value(valuationDate, liborCurve, model)

11368.236013739108

In [26]:
flr.value(valuationDate, liborCurve, model)

832.3997079027804

In [27]:
cap.printLeg()

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
OPTION TYPE FinLiborCapFloorTypes.CAP
STRIKE (%): 2.0
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE WED 6 JUN 2018
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
 FRI 8 JUN 2018          -         -            -     1.000000            -            -
MON 10 SEP 2018  0.2555556   2.34863       885.63     0.994034       885.63       885.63
MON 10 DEC 2018  0.2500000   2.28739       710.13     0.988382       759.95      1645.58
 FRI 8 MAR 2019  0.2444444   2.28951       695.58     0.982881       813.27      2458.85
MON 10 JUN 2019  0.2555556   3.03491      2579.49     0.975316      2593.01      5051.86
 MON 9 SEP 2019  0.2472222   2.60810      1456.85     0.969068      1558.24      6610.10
 MON 9 DEC 2019  0.2500000   2.57912      1394.03     0.962860      1544.48      8154.59
 MON 9 MAR 2020  0.2500000   2.57912      1385.10     0.95669

# Alternative Models

## Shifted Black

Shifted Black gives the same pdf at F+S as Black does at F. So if we want to have the PDF for F=0.25 at -0.25 because rates are negative then you need to set F=-0.50.

In [28]:
model = FinModelBlackShifted(0.25, -0.005)

In [29]:
cap.value(valuationDate, liborCurve, model)

11899.382559121314

In [30]:
flr.value(valuationDate, liborCurve, model)

1363.5496012117658

The floor has increased in value as the downside risk is greater.

## SABR Model

In [31]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573

In [32]:
model = FinModelSABR(alpha, beta, rho, nu)

In [33]:
cap.value(valuationDate, liborCurve, model)

11469.974835013834

In [34]:
flr.value(valuationDate, liborCurve, model)

934.1385291775069

## Shifted SABR Model

In [35]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573; shift = -0.005

In [36]:
model = FinModelSABRShifted(alpha, beta, rho, nu, shift)

In [37]:
cap.value(valuationDate, liborCurve, model)

11836.780832868084

In [38]:
flr.value(valuationDate, liborCurve, model)

1300.9445270317583

## Hull White Model

In [39]:
sigma = 0.01; alpha = 0.005

In [40]:
model = FinModelRatesHW(sigma, alpha)

In [41]:
cap.value(valuationDate, liborCurve, model)

14289.870253082552

In [42]:
flr.value(valuationDate, liborCurve, model)

2777.49553299532

Copyright (c) 2020 Dominic O'Kane